In [2]:
import pandas as pd
import numpy as np 
from itertools import permutations

In [3]:
#generate random data 
data = np.random.randint(10, size=(4, 2))
labels = ['A', 'B', 'C','D']
df = pd.DataFrame(data, columns=['X', 'Y'], index=labels)

#convert to a dictionary 
data_dict = {}
for idx, col in df.iterrows(): 
    data_dict[idx] = col.values
    
#data_dict   

In [4]:
#calculate the manhattan distance of our data values + put it into a n x n distance matrix 
def manhattan_dist(x, y):
    return sum(abs(a-b) for a,b in zip(x,y))

In [5]:
#put distances and labels into a dictionary
def build_tree():
    D = {}
    for label, val in data_dict.items():
        D[label] = {}
        for label2, val2 in data_dict.items():
            D[label][label2] = manhattan_dist(val, val2) 
    return D

In [6]:
tree = build_tree()
tree

{'A': {'A': 0, 'B': 6, 'C': 4, 'D': 5},
 'B': {'A': 6, 'B': 0, 'C': 10, 'D': 11},
 'C': {'A': 4, 'B': 10, 'C': 0, 'D': 1},
 'D': {'A': 5, 'B': 11, 'C': 1, 'D': 0}}

In [13]:
#visualize as n x n distance matrix         
dm = pd.DataFrame.from_dict(tree)
dm

,A,B,C,D
A,0,6,4,5
B,6,0,10,11
C,4,10,0,1
D,5,11,1,0


In [14]:
#check if the matrix is additive using four point condition
def is_additive(matrix, labels):
    i = labels[0]
    j = labels[1] 
    k = labels[2] 
    l = labels[3] 
    result = ''
    if ((matrix[i][j] + matrix[k][l] <= matrix[i][k] + matrix[j][l]) and 
    (matrix[i][k] + matrix[j][l] == matrix[i][l] + matrix[j][k])):
        result = "Additive"
    else:
        result = "Not additive"
    return result 

In [15]:
cols = 'ABCD'
labels = permutations(temp,4)

NameError: name 'temp' is not defined

In [16]:
check = []
for combs in labels:
    new_label = []
    for item in combs:
        new_label+=item
    check.append(is_additive(dm, new_label))
check

IndexError: list index out of range

## Niegbor join

In [17]:
import copy
clusters = []
for cluster in list(dm):
    clusters.append(cluster)
cluster2idx = { name : idx for idx, name in enumerate(clusters) }
idx2cluster = copy.deepcopy(clusters)
m = copy.deepcopy(dm).to_dict()
Z = np.empty((0,4), float)
result = ""
print(clusters)
#membership = [set([cluster2idx[name]]) for name in clusters]

['A', 'B', 'C', 'D']


In [18]:
M, N = dm.shape
new_cluster_idx = M+1
while len(clusters) > 1:
    print(result)
    print(m)
    idx1 = None
    idx2 = None
    ui = 0
    uj = 0
    Sd = 100000
    for i in clusters:
        for j in clusters:
            if i != j:
                ui = sum(m[i][k] for k in clusters)
                uj = sum(m[j][k] for k in clusters)
                curr = (M - 2) * m[i][j] - ui - uj
                if Sd > curr:
                    Sd = curr
                    idx1 = cluster2idx[i]
                    idx2 = cluster2idx[j]
    
    Dij = m[idx2cluster[idx1]][idx2cluster[idx2]]
    new_cluster = "(" + idx2cluster[idx1] +"." + idx2cluster[idx2] + ")"
    result = new_cluster
    m[new_cluster] = {}
    new_cluster_idx += 1
    #membership.append(membership[cluster2idx[idx1]] | membership[cluster2idx[idx2]])
    #Z = np.append(Z, np.array([[idx1, idx2, Sd]]))
    
    m[idx2cluster[idx1]][new_cluster] = (m[idx2cluster[idx1]][idx2cluster[idx2]] + (1/(M-2)) * (ui - uj))/2
    m[idx2cluster[idx2]][new_cluster] = (m[idx2cluster[idx1]][idx2cluster[idx2]] + (1/(M-2)) * (uj - ui))/2
    
    #if idx2cluster[idx1] in m: del m[idx2cluster[idx1]]
    #if idx2cluster[idx2] in m: del m[idx2cluster[idx2]]
    
    clusters.remove(idx2cluster[idx1])
    clusters.remove(idx2cluster[idx2])
    
    m[new_cluster][new_cluster] = 0
    for c in clusters:
        m[c][new_cluster] = (m[c][idx2cluster[idx1]] + m[c][idx2cluster[idx2]] - Dij)/2
        m[new_cluster][c] = m[c][new_cluster]
        for k in list(m[c].keys()):
            if k == idx2cluster[idx1] or k == idx2cluster[idx2]:
                del m[c][k]
        
    del cluster2idx[idx2cluster[idx1]]
    del cluster2idx[idx2cluster[idx2]]
    if idx2cluster[idx1] in m: del m[idx2cluster[idx1]]
    if idx2cluster[idx2] in m: del m[idx2cluster[idx2]]
    
    clusters.append(new_cluster)
    cluster2idx[new_cluster] = len(idx2cluster)
    idx2cluster.append(new_cluster)
        
    #M -= 1
    #N -= 1


{'A': {'A': 0, 'B': 6, 'C': 4, 'D': 5}, 'B': {'A': 6, 'B': 0, 'C': 10, 'D': 11}, 'C': {'A': 4, 'B': 10, 'C': 0, 'D': 1}, 'D': {'A': 5, 'B': 11, 'C': 1, 'D': 0}}
(A.B)
{'C': {'C': 0, 'D': 1, '(A.B)': 4.0}, 'D': {'C': 1, 'D': 0, '(A.B)': 5.0}, '(A.B)': {'(A.B)': 0, 'C': 4.0, 'D': 5.0}}
(C.D)
{'(A.B)': {'(A.B)': 0, '(C.D)': 4.0}, '(C.D)': {'(C.D)': 0, '(A.B)': 4.0}}


In [19]:
print(result)

((A.B).(C.D))
